This cell block is for importing all the required libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import levene
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from scipy.stats import ttest_ind

In [ ]:
# Mount your Google Drive (authentication required only for the first time)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Specify the path to your CSV file on Google Drive (replace with your actual path)
data_path = '/content/gdrive/MyDrive/HRI Project CSV Data/Test Score and Evaluation Score Data - Sheet1.csv'

# Read the data from the CSV file
data = pd.read_csv(data_path)

This cell block contains code to perfoerm the ANOVA test

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
import pandas as pd

# Assuming 'data' is your DataFrame containing the dataset

# Fit the ANOVA model with ChangeInTestScore as the target variable and LearningProcess and Gender as the grouping variables:
model = smf.ols("ChangeInTestScore ~ C(LearningProcess) * C(Gender)", data=data).fit()

# Perform ANOVA with type 2 tests for mixed effects:
results_anova = anova_lm(model, typ=2)

# Get the total sum of squares (SS):
ss_total = results_anova['sum_sq'].sum()

# Calculate eta-squared (η²):
eta_squared = results_anova['sum_sq'] / ss_total

# Calculate partial eta-squared (η²_p):
df_group = results_anova['df'][0]  # Degrees of freedom for the group variable
df_residual = results_anova['df'][1]  # Degrees of freedom for the residual
eta_squared_partial = (results_anova['sum_sq'] / (results_anova['sum_sq'] + df_residual * model.mse_resid)) * df_group

# Combine ANOVA results with eta-squared and partial eta-squared:
results_combined = pd.concat([results_anova, eta_squared.rename('eta_squared'), eta_squared_partial.rename('partial_eta_squared')], axis=1)

# Print the combined results:
print(results_combined)

                                  sum_sq    df         F    PR(>F)  \
C(LearningProcess)             38.285714   2.0  1.501401  0.254468   
C(Gender)                       2.571429   1.0  0.201681  0.659788   
C(LearningProcess):C(Gender)   10.464286   2.0  0.410364  0.670635   
Residual                      191.250000  15.0       NaN       NaN   

                              eta_squared  partial_eta_squared  
C(LearningProcess)               0.157833             1.500350  
C(Gender)                        0.010601             0.335664  
C(LearningProcess):C(Gender)     0.043139             0.901538  
Residual                         0.788428             1.875000  


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
import pandas as pd

# Assuming 'data' is your DataFrame containing the dataset

# Fit the ANOVA model with ChangeInTestScore as the target variable and LearningProcess, Gender, and their interaction as the grouping variables:
model = smf.ols("ChangeInTestScore ~ C(LearningProcess) + C(Gender) + C(LearningProcess):C(Gender)", data=data).fit()

# Print the model summary:
print(model.summary())

# Perform ANOVA with type 2 tests for mixed effects:
results_anova = anova_lm(model, typ=2)

# Get the total sum of squares (SS):
ss_total = results_anova['sum_sq'].sum()

# Calculate eta-squared (η²):
eta_squared = results_anova['sum_sq'] / ss_total

# Calculate partial eta-squared (η²_p):
df_group = results_anova['df'][0]  # Degrees of freedom for the group variable
df_residual = results_anova['df'][1]  # Degrees of freedom for the residual
eta_squared_partial = (results_anova['sum_sq'] / (results_anova['sum_sq'] + df_residual * model.mse_resid)) * df_group

# Combine ANOVA results with eta-squared and partial eta-squared:
results_combined = pd.concat([results_anova, eta_squared.rename('eta_squared'), eta_squared_partial.rename('partial_eta_squared')], axis=1)

# Print the combined results:
print(results_combined)

                            OLS Regression Results                            
Dep. Variable:      ChangeInTestScore   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                 -0.057
Method:                 Least Squares   F-statistic:                    0.7841
Date:                Sun, 17 Mar 2024   Prob (F-statistic):              0.577
Time:                        12:05:38   Log-Likelihood:                -52.993
No. Observations:                  21   AIC:                             118.0
Df Residuals:                      15   BIC:                             124.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [ ]:
# Assuming 'data' is your DataFrame containing the independent variables (LearningProcess and Gender)

# Convert categorical variables to one-hot encoded format
data_encoded = pd.get_dummies(data, columns=['LearningProcess', 'Gender'])

# Calculate the correlation matrix
correlation_matrix = data_encoded.corr()

# Print the correlation matrix
print(correlation_matrix)


                                ChangeInTestScore  \
ChangeInTestScore                        1.000000   
LearningProcessEvaluationScore           0.031544   
LearningProcess_ALL                     -0.377517   
LearningProcess_HLL                      0.099346   
LearningProcess_RLL                      0.278170   
Gender_Female                            0.071640   
Gender_Male                             -0.071640   

                                LearningProcessEvaluationScore  \
ChangeInTestScore                                 3.154371e-02   
LearningProcessEvaluationScore                    1.000000e+00   
LearningProcess_ALL                              -9.331128e-17   
LearningProcess_HLL                              -2.801578e-01   
LearningProcess_RLL                               2.801578e-01   
Gender_Female                                    -1.942545e-01   
Gender_Male                                       1.942545e-01   

                                LearningProces

In [ ]:
# Separate data for each teaching method
teaching_method1_data = data[data['TeachingMethod'] == 'Application']['ScoreChange']
teaching_method2_data = data[data['TeachingMethod'] == 'Human Tutor']['ScoreChange']
teaching_method3_data = data[data['TeachingMethod'] == 'NAO']['ScoreChange']

# Levene's test (replace with actual method names)
w_statistic, p_value = levene(teaching_method1_data, teaching_method2_data, teaching_method3_data)

print(f"Levene's test statistic: {w_statistic}")
print(f"p-value: {p_value}")

# Interpretation
if p_value > 0.05:
    print("Fail to reject the null hypothesis of equal variances.")
else:
    print("Reject the null hypothesis of equal variances.")

Levene's test statistic: 0.8913738019169329
p-value: 0.4274379268498227
Fail to reject the null hypothesis of equal variances.


In [ ]:
# Separate features (including the group variable) and the target variable
features = data.drop('ScoreChange', axis=1)
target_variable = data['ScoreChange']

groups = np.unique(features['TeachingMethod'])  # Get unique group labels

# Perform pairwise t-tests with Bonferroni correction
for i in range(len(groups) - 1):
  for j in range(i + 1, len(groups)):
    _, p_value = ttest_ind(target_variable[features['TeachingMethod'] == groups[i]],
                           target_variable[features['TeachingMethod'] == groups[j]])
    alpha_adjusted = 0.05 / (len(groups) * (len(groups) - 1) / 2)  # Bonferroni correction
    if p_value < alpha_adjusted:
      print(f"Significant difference between {groups[i]} and {groups[j]} (p-value: {p_value:.4f})")
    else:
      print(p_value)
      print(alpha_adjusted)

0.295539594516705
0.016666666666666666
0.0839899482776256
0.016666666666666666
0.6032164494297924
0.016666666666666666
